<div><img style="float: right; width: 120px; vertical-align:middle" src="https://www.upm.es/sfs/Rectorado/Gabinete%20del%20Rector/Logos/EU_Informatica/ETSI%20SIST_INFORM_COLOR.png" alt="ETSISI logo" />


# Clasificación de texto con RNN<a id="top"></a>

<i><small>Última actualización: 2023-05-11</small></i></div>

***

## Introducción

En un _notebook_ anterior exploramos la clasificación de textos con CNN, las cuales son adecuadas para capturar dependencias locales en datos de texto. Sin embargo, a veces necesitamos un modelo más potente que pueda capturar dependencias a largo plazo en los datos. Aquí es donde entran en juego las RNN.

Las RNN están diseñadas específicamente para modelar datos secuenciales, lo que las hace ideales para tareas de clasificación de texto. A diferencia de las redes neuronales tradicionales, que procesan las entradas independientemente unas de otras, las RNN mantienen una memoria de las entradas anteriores y utilizan esta información para hacer predicciones sobre la entrada actual.

## Objetivos

Vamos a explorar cómo utilizar RNN para la clasificación de texto en el mismo problema que en el _notebook_ donde clasificábamos las reseñas de Amazon que los usuarios hicieron sobre los productos mediante CNN, pero esta vez con RNN.

Veremos que en realidad los cambios son mínimos, ya que es poco más que cambiar una capa por otra.

## Bibliotecas y configuración

A continuación importaremos las bibliotecas que se utilizarán a lo largo del cuaderno.

In [ ]:
import os.path
import requests
from shutil import unpack_archive

import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt

También configuraremos algunos parámetros para adaptar la presentación gráfica.

In [ ]:
plt.style.use('ggplot')
plt.rcParams.update({'figure.figsize': (20, 6),'figure.dpi': 64})

Y crearemos los directorios necesarios en caso de que no se hayan creado previamente

In [ ]:
os.makedirs('tmp', exist_ok=True)

***

## Parámetros comunes

Mantendremos los mismos parámetros globales para poder comparar ambos métodos.

In [ ]:
# Cuántas dimensiones tienen nuestros vectores de palabras (usaremos
# GloVe, así que 50, 100, 200 o 300)
EMBEDDING_DIM = 300
# Tamaño máximo de nuestro vocabulario (se elegirán los token más
# frecuentes)
MAX_VOCAB_SIZE = 16384
# Longitud máxima de las secuencias de palabras
MAX_SEQUENCE_LEN = 32

## Preprocesamiento de datos

El proceso que llevaremos a cabo será el mismo que hicimos en el _notebook_ anterior.

### Descarga del _dataset_ ...

In [ ]:
DATASET_URL = 'https://jmcauley.ucsd.edu/data/amazon_v2/categoryFilesSmall/Digital_Music_5.json.gz'
DATASET_ZIP = 'tmp/Digital_Music_5.json.gz'

# Download the remote file if it does not exist
if not os.path.exists(DATASET_ZIP):
    with open(DATASET_ZIP, 'wb') as f:
        print(f'Downloading {DATASET_ZIP}...')
        r = requests.get(DATASET_URL, verify=False)
        f.write(r.content)
        print('OK')

corpus = pd.read_json(DATASET_ZIP, lines=True)
corpus.dropna(subset=['overall', 'reviewText'], inplace=True)
corpus.head()

### ... preparación de las entradas ...

In [ ]:
x_train = corpus['reviewText'].astype(str).str.strip()
print(f'Training input shape: {x_train.shape}')

### ... de las salidas ...

In [ ]:
y_train = corpus['overall'].astype(int).replace({
    1: 0,
    2: 0,
    3: 1,
    4: 2,
    5: 2,
})
print(f'Training output shape: {y_train.shape}')

### ... la capa de vectorización de texto (_TextVectorization_) ...

In [ ]:
vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=MAX_VOCAB_SIZE + 2,
    output_sequence_length=MAX_SEQUENCE_LEN,
    name='vectorization',
)
vectorize_layer.adapt(x_train)

print(f'Vocabulary length: {len(vectorize_layer.get_vocabulary())}')

### ... los _embeddings_ preentrenados de GloVe ...

In [ ]:
GLOVE_URL = 'http://nlp.stanford.edu/data/glove.6B.zip'
GLOVE_FILE = 'tmp/glove.6B.zip'

# Download the compressed GloVe dataset (if you don't already have it)
if not os.path.exists(GLOVE_FILE):
    print('Downloading ...', end='')
    with open(GLOVE_FILE, 'wb') as f:
        r = requests.get(GLOVE_URL, allow_redirects=True)
        f.write(r.content)
    print('OK')

# Unzip it in the directory 'glove'.
print('Unpacking ...', end='')
unpack_archive(GLOVE_FILE, 'tmp')
print('OK')

### ... cogiendo los pesos de las `MAX_VOCAB_SIZE` palabras más comunes ...

In [ ]:
print(f'Loading GloVe {EMBEDDING_DIM}-d embedding... ', end='')
word2vec = {}
with open(f'tmp/glove.6B.{EMBEDDING_DIM}d.txt') as f:
    for line in f:
        word, vector = line.split(maxsplit=1)
        word2vec[word] = np.fromstring(vector,'f', sep=' ')
print(f'done ({len(word2vec)} word vectors loaded)')

print('Creating embedding matrix with GloVe vectors... ', end='')
# Our newly created embedding: a matrix of zeros
embedding_matrix = np.zeros((MAX_VOCAB_SIZE + 2, EMBEDDING_DIM))

ko_words = 0
for i, word in enumerate(vectorize_layer.get_vocabulary()):
    if word == '[UNK]':
        # The second word is for an unknown token, in glove is 'unk'
        word = 'unk'

    # Get the word vector and overwrite the row in its corresponding position
    word_vector = word2vec.get(word)
    if word_vector is not None:
        embedding_matrix[i] = word_vector
    else:
        ko_words += 1
print(f'done ({ko_words} words unassigned)')

### ... y cargándolos dentro de la capa de `Embedding`

In [ ]:
embedding_layer = tf.keras.layers.Embedding(
    input_dim=embedding_matrix.shape[0],
    output_dim=embedding_matrix.shape[1],
    weights=[embedding_matrix],
    input_length=MAX_SEQUENCE_LEN,
    trainable=False,
    name='Embedding',
)

## Clasificación basada en redes neuronales recurrentes

Y ahora, en lugar de utilizar CNN, utilizaremos RNN. En este caso, el conjunto de dimensiones lo realizan la capa `TextVectorization` (que convierte el texto en secuencias de enteros de longitud $T$) y la capa Embedding (que convierte cada entero en un vector de dimensiones $D$), convirtiendo la entrada en un tensor con la forma $N \times T \times D$.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(1,), dtype=tf.string),
    vectorize_layer,
    embedding_layer,
    tf.keras.layers.GRU(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='sigmoid')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['sparse_categorical_accuracy'],
)
model.summary()

Aunque el número de parámetros es similar, aumentar el número de unidades en una unidad recurrente no aumenta mucho el número de parámetros en nuestro modelo. Sin embargo, sí que aumentará mucho el tiempo de entrenamiento. Por lo tanto, nuestro modelo no podrá obtener resultados tan buenos como los anteriores.

Entrenemos el modelo y esperemos que todo vaya bien (otra vez)

In [ ]:
history = model.fit(x_train, y_train, epochs=5)

Echemos un vistazo al progreso del entrenamiento:

In [ ]:
pd.DataFrame(history.history).plot()
plt.yscale('log')
plt.xlabel('Epoch num.')
plt.show()

Veamos ahora cómo interpreta el sentimiento de una reseña buena, regular y mala extraída del sitio web de amazon.

In [ ]:
good = "My nephew is on the autism spectrum and likes to fidget with things so I knew this toy would be a hit. Was concerned that it may not be \"complex\" enough for his very advanced brain but he really took to it. Both him (14 yrs) and his little brother (8 yrs) both enjoyed playing with it throughout Christmas morning. I'm always happy when I can find them something unique and engaging."
poor = "I wasn't sure about this as it's really small. I bought it for my 9 year old grandson. I was ready to send it back but my daughter decided it was a good gift so I'm hoping he likes it. Seems expensive for the price though to me."
evil = "I just wanted to follow up to say that I reported this directly to the company and had no response. I have not gotten any response from my review. The level of customer service goes a long way when an item you purchase is defective and this company didn’t care to respond. No I am even more Leary about ordering anything from this company. I never asked for a refund or replacement since I am not able to return it. I’m just wanted to let them know that this was a high dollar item and I expected it to be a quality item. Very disappointed! I bought this for my grandson for Christmas. He loved it and played with it a lot. My daughter called to say that the stickers were peeling on the corners. I am not able to take it from my grandson because he is autistic and wouldn’t understand. I just wanted to warn others who are wanting to get this. Please know that this is a cool toy and it may not happen to yours so it is up to you."

probabilities = model.predict([good, poor, evil], verbose=0)
print(f'Good was classified as {np.argmax(probabilities[0])}')
print(f'Poor was classified as {np.argmax(probabilities[1])}')
print(f'Evil was classified as {np.argmax(probabilities[2])}')

## Conclusiones

Hemos estudiado cómo utilizar RNN para clasificar textos y las hemos comparado con las CNN. Aunque ambas arquitecturas pueden ser eficaces para la clasificación de textos, presentan algunas diferencias clave.

La principal es las RNN son más adecuadas para captar las dependencias a largo plazo en los datos de texto, mientras que las CNN son más adecuadas para captar las dependencias locales. Esto hace que las RNN sean una buena opción para tareas como el análisis de sentimientos o la traducción de idiomas, donde el contexto de una palabra o frase puede ser crucial para determinar su significado.

Sin embargo, el entrenamiento de las RNN puede ser más costoso que el de las CNN (ya hemos visto el tiempo que tarda una red pequeña en ser entrenada durante 5 _epochs_). Por otro lado, las CNN son más rápidas de entrenar y se adaptan mejor a conjuntos de datos más grandes. Son especialmente adecuadas para tareas como la clasificación de textos o el reconocimiento de imágenes, en las que las características locales son importantes.

***

<div><img style="float: right; width: 120px; vertical-align:top" src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" alt="Creative Commons by-nc-sa logo" />

[Volver al inicio](#top)

</div>